In [1]:
import scipy.io
import numpy as np
import random
import os

In [2]:
person_nums = [1,2,4,5,6,8,9,10,11,12,13,17,19,21,22,25,26,27,28,29]
## Build the baseline model for emotion recognition with dropout layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, SimpleRNN, LSTM, Conv2D, Flatten, MaxPooling2D, GRU, AveragePooling2D, Dropout, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from feature_emotion import feature_extract, split_data, label_unique_tuples, split_data_unknown, split_data_few_shot
gts, sensor_nums, walk_nums, trace_nums, people_nums, spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features = feature_extract(person_nums)

walk_nums_all = np.squeeze(walk_nums)
trace_nums_all = np.squeeze(trace_nums)
people_nums_all = np.squeeze(people_nums)
ra = 0.1
test_person_id = [25]
flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

2023-12-04 21:19:50.079702: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


35261
Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_30.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2023-12-04 21:21:12.750283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 21:21:12.761640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 21:21:12.761876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Test MAE:
1.6912948746448275
[1.64245941 1.65183379]
1.6471465990461152


In [4]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_31.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:21:29.896959: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_5/kernel/Assign' id:15548 op device:{requested: '', assigned: ''} def:{{{node dense_5/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5/kernel, dense_5/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:21:31.202623: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_55/lstm_cell_55/kernel/m/Assign' id:16469 op device:{requested: '', assigned: ''} def:{{{node lstm_55/lstm_cell_55/kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_55/lstm_cell_55/kernel/m, lstm_55/lstm_cell_55/kernel/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a se

Test MAE:
1.7022901228104514
[1.6715914  1.64038013]
1.65598576562158


In [5]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_32.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:21:45.139047: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_80/lstm_cell_80/bias/Assign' id:19807 op device:{requested: '', assigned: ''} def:{{{node lstm_80/lstm_cell_80/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_80/lstm_cell_80/bias, lstm_80/lstm_cell_80/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:21:46.612886: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_87/lstm_cell_87/recurrent_kernel/v/Assign' id:26273 op device:{requested: '', assigned: ''} def:{{{node lstm_87/lstm_cell_87/recurrent_kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_87/lstm_cell_87/recurrent_kernel/v, lstm_87/lstm_cell_87/recurrent_kernel/v/Ini

Test MAE:
1.711123670894524
[1.67279608 1.65561495]
1.6642055133293414


In [6]:
ra = 0.2

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

35453


In [7]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_33.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:22:06.212869: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_118/lstm_cell_118/bias/Assign' id:29198 op device:{requested: '', assigned: ''} def:{{{node lstm_118/lstm_cell_118/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_118/lstm_cell_118/bias, lstm_118/lstm_cell_118/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:22:08.031955: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_127/lstm_cell_127/bias/v/Assign' id:35554 op device:{requested: '', assigned: ''} def:{{{node lstm_127/lstm_cell_127/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_127/lstm_cell_127/bias/v, lstm_127/lstm_cell_127/bias/v/Initializer/zeros)}}' was changed b

Test MAE:
1.6081600182663254
[1.68050465 1.459683  ]
1.5700938248819158


In [8]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_34.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:22:21.982616: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_158/lstm_cell_158/recurrent_kernel/Assign' id:38900 op device:{requested: '', assigned: ''} def:{{{node lstm_158/lstm_cell_158/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_158/lstm_cell_158/recurrent_kernel, lstm_158/lstm_cell_158/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:22:24.183120: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_16/bias/v/Assign' id:45097 op device:{requested: '', assigned: ''} def:{{{node dense_16/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_16/bias/v, dense_16/bias/v/Initializer/zeros)}}' was changed by setting

Test MAE:
1.641224082549721
[1.72558247 1.48105512]
1.6033187993737155


In [9]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_35.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:22:38.763891: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_23/kernel/Assign' id:52514 op device:{requested: '', assigned: ''} def:{{{node dense_23/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_23/kernel, dense_23/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:22:41.322959: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_22/kernel/m/Assign' id:53103 op device:{requested: '', assigned: ''} def:{{{node conv2d_22/kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_22/kernel/m, conv2d_22/kernel/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect

Test MAE:
1.6497477389275257
[1.69828373 1.54097142]
1.6196275773898576


In [10]:
ra = 0.3

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

35645


In [11]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_36.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:23:02.149619: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_235/lstm_cell_235/kernel/Assign' id:57822 op device:{requested: '', assigned: ''} def:{{{node lstm_235/lstm_cell_235/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_235/lstm_cell_235/kernel, lstm_235/lstm_cell_235/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:23:05.146266: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_234/lstm_cell_234/recurrent_kernel/m/Assign' id:62539 op device:{requested: '', assigned: ''} def:{{{node lstm_234/lstm_cell_234/recurrent_kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_234/lstm_cell_234/recurrent_kernel/m, lstm_234/lstm

Test MAE:
1.5917399408718829
[1.59155541 1.49720813]
1.5443817714674284


In [12]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_37.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:23:20.295987: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_262/lstm_cell_262/kernel/Assign' id:65453 op device:{requested: '', assigned: ''} def:{{{node lstm_262/lstm_cell_262/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_262/lstm_cell_262/kernel, lstm_262/lstm_cell_262/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:23:23.658618: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_295/lstm_cell_295/bias/v/Assign' id:72778 op device:{requested: '', assigned: ''} def:{{{node lstm_295/lstm_cell_295/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_295/lstm_cell_295/bias/v, lstm_295/lstm_cell_295/bias/v/Initializer/zeros)}

Test MAE:
1.5645481466279971
[1.57664321 1.46556027]
1.5211017363894301


In [13]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_38.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:23:39.653486: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_313/lstm_cell_313/recurrent_kernel/Assign' id:76944 op device:{requested: '', assigned: ''} def:{{{node lstm_313/lstm_cell_313/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_313/lstm_cell_313/recurrent_kernel, lstm_313/lstm_cell_313/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:23:43.394431: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_323/lstm_cell_323/kernel/v/Assign' id:81864 op device:{requested: '', assigned: ''} def:{{{node lstm_323/lstm_cell_323/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_323/lstm_cell_323/kernel/v, lstm_323/lst

Test MAE:
1.5690708481280853
[1.61052762 1.45481653]
1.5326720729338383


In [14]:
ra = 0.4

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

35837


In [15]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_39.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:24:05.514958: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_347/lstm_cell_347/kernel/Assign' id:85675 op device:{requested: '', assigned: ''} def:{{{node lstm_347/lstm_cell_347/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_347/lstm_cell_347/kernel, lstm_347/lstm_cell_347/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:24:10.008702: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_344/lstm_cell_344/kernel/v/Assign' id:90855 op device:{requested: '', assigned: ''} def:{{{node lstm_344/lstm_cell_344/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_344/lstm_cell_344/kernel/v, lstm_344/lstm_cell_344/kernel/v/Initializer

Test MAE:
1.4222424305284147
[1.46807638 1.27316245]
1.3706194162368774


In [16]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_40.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:24:26.154314: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_388/lstm_cell_388/recurrent_kernel/Assign' id:95566 op device:{requested: '', assigned: ''} def:{{{node lstm_388/lstm_cell_388/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_388/lstm_cell_388/recurrent_kernel, lstm_388/lstm_cell_388/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:24:30.727997: W tensorflow/c/c_api.cc:304] Operation '{name:'beta_2_10/Assign' id:99221 op device:{requested: '', assigned: ''} def:{{{node beta_2_10/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](beta_2_10, beta_2_10/Initializer/initial_value)}}' was changed by setting attribute after

Test MAE:
1.4101446881327022
[1.42168468 1.28572955]
1.3537071156747562


In [17]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_41.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:24:47.755528: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_441/lstm_cell_441/bias/Assign' id:107368 op device:{requested: '', assigned: ''} def:{{{node lstm_441/lstm_cell_441/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_441/lstm_cell_441/bias, lstm_441/lstm_cell_441/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:24:52.759012: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_44/bias/m/Assign' id:108472 op device:{requested: '', assigned: ''} def:{{{node conv2d_44/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_44/bias/m, conv2d_44/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session.

Test MAE:
1.456522571159802
[1.51231454 1.31364319]
1.4129788605208249


In [18]:
ra = 0.5

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

36029


In [19]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_42.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:25:16.350776: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_452/lstm_cell_452/bias/Assign' id:112437 op device:{requested: '', assigned: ''} def:{{{node lstm_452/lstm_cell_452/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_452/lstm_cell_452/bias, lstm_452/lstm_cell_452/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:25:22.220757: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_446/lstm_cell_446/kernel/m/Assign' id:117770 op device:{requested: '', assigned: ''} def:{{{node lstm_446/lstm_cell_446/kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_446/lstm_cell_446/kernel/m, lstm_446/lstm_cell_446/kernel/m/Initializer/zeros)}}' was

Test MAE:
1.4053288926926168
[1.41048846 1.30483286]
1.3576606615090077


In [20]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_43.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:25:39.682568: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_514/lstm_cell_514/recurrent_kernel/Assign' id:125661 op device:{requested: '', assigned: ''} def:{{{node lstm_514/lstm_cell_514/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_514/lstm_cell_514/recurrent_kernel, lstm_514/lstm_cell_514/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:25:45.487991: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_55/bias/v/Assign' id:127609 op device:{requested: '', assigned: ''} def:{{{node conv2d_55/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_55/bias/v, conv2d_55/bias/v/Initializer/zeros)}}' was changed by s

Test MAE:
1.39999409909111
[1.42497978 1.29421109]
1.3595954371087344


In [21]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_44.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:26:03.625638: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_550/lstm_cell_550/kernel/Assign' id:134712 op device:{requested: '', assigned: ''} def:{{{node lstm_550/lstm_cell_550/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_550/lstm_cell_550/kernel, lstm_550/lstm_cell_550/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:26:09.811050: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_57/kernel/v/Assign' id:137414 op device:{requested: '', assigned: ''} def:{{{node dense_57/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_57/kernel/v, dense_57/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it w

Test MAE:
1.452734310072636
[1.45802416 1.38718185]
1.422603005244408


In [22]:
ra = 0.6

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

36221


In [23]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_45.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:26:34.636716: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_576/lstm_cell_576/bias/Assign' id:142210 op device:{requested: '', assigned: ''} def:{{{node lstm_576/lstm_cell_576/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_576/lstm_cell_576/bias, lstm_576/lstm_cell_576/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:26:41.208222: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_573/lstm_cell_573/kernel/m/Assign' id:145703 op device:{requested: '', assigned: ''} def:{{{node lstm_573/lstm_cell_573/kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_573/lstm_cell_573/kernel/m, lstm_573/lstm_cell_573/kernel/m/Initializer/zeros)}}' was

Test MAE:
1.5544186666989939
[1.54348724 1.47565667]
1.5095719557542067


In [24]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_46.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:26:59.982635: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_66/kernel/Assign' id:154034 op device:{requested: '', assigned: ''} def:{{{node dense_66/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_66/kernel, dense_66/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:27:07.496351: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_65/bias/m/Assign' id:154639 op device:{requested: '', assigned: ''} def:{{{node conv2d_65/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_65/bias/m, conv2d_65/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and 

Test MAE:
1.5128105252981185
[1.46518753 1.45309638]
1.4591419586768517


In [25]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_47.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:27:27.121729: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_631/lstm_cell_631/recurrent_kernel/Assign' id:157623 op device:{requested: '', assigned: ''} def:{{{node lstm_631/lstm_cell_631/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_631/lstm_cell_631/recurrent_kernel, lstm_631/lstm_cell_631/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:27:34.496022: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_663/lstm_cell_663/bias/v/Assign' id:165058 op device:{requested: '', assigned: ''} def:{{{node lstm_663/lstm_cell_663/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_663/lstm_cell_663/bias/v, lstm_663/lstm_ce

Test MAE:
1.4926363005087926
[1.4338164  1.42623781]
1.4300271070920505


In [26]:
from feature_emotion import feature_extract, split_data, label_unique_tuples, split_data_unknown
gts, sensor_nums, walk_nums, trace_nums, people_nums, spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features = feature_extract(person_nums)

walk_nums_all = np.squeeze(walk_nums)
trace_nums_all = np.squeeze(trace_nums)
people_nums_all = np.squeeze(people_nums)

flag_tr_val_te = split_data_unknown(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

35069


In [27]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_p"+str(test_person_id[0])+'_20.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:28:46.033909: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1_1/bias/Assign' id:172480 op device:{requested: '', assigned: ''} def:{{{node dense_1_1/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_1_1/bias, dense_1_1/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:28:53.859547: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_2_1/bias/m/Assign' id:173111 op device:{requested: '', assigned: ''} def:{{{node conv2d_2_1/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_2_1/bias/m, conv2d_2_1/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trig

Test MAE:
1.8082893202949015
[1.75361856 1.75440861]
1.754013585007709


In [28]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_p"+str(test_person_id[0])+'_21.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:29:21.569788: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_2_2/lstm_cell_705/kernel/Assign' id:176065 op device:{requested: '', assigned: ''} def:{{{node lstm_2_2/lstm_cell_705/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_2_2/lstm_cell_705/kernel, lstm_2_2/lstm_cell_705/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:29:29.746598: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_34_2/lstm_cell_737/bias/m/Assign' id:182877 op device:{requested: '', assigned: ''} def:{{{node lstm_34_2/lstm_cell_737/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_34_2/lstm_cell_737/bias/m, lstm_34_2/lstm_cell_737/bias/m/Initializer/z

Test MAE:
1.8782135237077748
[1.78813388 1.84819017]
1.8181620251317943


In [29]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_p"+str(test_person_id[0])+'_22.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 21:29:57.870954: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_27_3/lstm_cell_767/kernel/Assign' id:189298 op device:{requested: '', assigned: ''} def:{{{node lstm_27_3/lstm_cell_767/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_27_3/lstm_cell_767/kernel, lstm_27_3/lstm_cell_767/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 21:30:06.478893: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_3_3/lstm_cell_743/bias/v/Assign' id:192286 op device:{requested: '', assigned: ''} def:{{{node lstm_3_3/lstm_cell_743/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_3_3/lstm_cell_743/bias/v, lstm_3_3/lstm_cell_743/bias/v/Initializer/z

Test MAE:
1.8360767780994036
[1.76225625 1.79604357]
1.7791499128993253
